In [50]:
from __future__ import absolute_import

import torch
from torch import nn
from torch.nn import functional as F
import torchvision

from aligned.HorizontalMaxPool2D import HorizontalMaxPool2d

__all__ = ['ResNet50']

class ResNet50(nn.Module):
    def __init__(self, num_classes, loss={'softmax'}, aligned=False, **kwargs):
        super().__init__()
        self.loss = loss
        resnet50 = torchvision.models.resnet50(pretrained=True)

        self.base = nn.Sequential(*list(resnet50.children())[:-2])  #源代码
        self.base2 = nn.Sequential(*list(resnet.children())[:5])  #源代码
        #self.base = nn.Sequential(*list(resnet50.children())[:])
        #self.base = nn.Sequential(*list(resnet50.children())[:])
        self.classifier = nn.Linear(2048, num_classes)
        self.feat_dim = 2048 # feature dimension
        self.aligned = aligned
        self.horizon_pool = HorizontalMaxPool2d()
        self.custom_layers = nn.Sequential(
            nn.Conv2d(256, 1024, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(1024, 128, kernel_size=3, stride=1, padding=1),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        if self.aligned:
            self.bn = nn.BatchNorm2d(2048)
            self.relu = nn.ReLU(inplace=True)
            self.conv1 = nn.Conv2d(2048, 128, kernel_size=1, stride=1, padding=0, bias=True)

    def forward(self, x):

        tmp=self.base2(x)
        x = self.base(x)
        if not self.training:
            tmp=self.custom_layers(tmp)
            tmp_merged = tmp.view(tmp.size(0), -1, tmp.size(-1))
            # 添加新的维度，使其形状为 (32, 2048, 8, 1)
            lf = tmp_merged.unsqueeze(-1)
            #lf = self.horizon_pool(x)
        if self.aligned and self.training:
            lf = self.bn(x)
            lf = self.relu(lf)
            lf = self.horizon_pool(lf)
            
            lf = self.conv1(lf)
        if self.aligned or not self.training:
            lf = lf.view(lf.size()[0:3])
            lf = lf / torch.pow(lf,2).sum(dim=1, keepdim=True).clamp(min=1e-12).sqrt()
        x = F.avg_pool2d(x, x.size()[2:])
        f = x.view(x.size(0), -1)#这一步是需要张平的，作用还是把（32,2048,1,1，）转换成（32，2048）
        #f = 1. * f / (torch.norm(f, 2, dim=-1, keepdim=True).expand_as(f) + 1e-12)#这一步是数据归一化

        if not self.training:#如果不是训练
            return f,lf

        y = self.classifier(f)
        if self.loss == {'softmax'}:
            return y
        elif self.loss == {'metric'}:
            if self.aligned: return  f, lf
            return f
        elif self.loss == {'softmax', 'metric'}:
            if self.aligned: return y, f, lf
            return y, f
        else:
            raise KeyError("Unsupported loss: {}".format(self.loss))



In [51]:
resnet50=ResNet50(751)
imgs=torch.Tensor(32,3,256,128)
resnet50.eval()
f=resnet50(imgs)
print(f)
torch.save(resnet50, "resnet50.pt")

(tensor([[0.1787, 0.0075, 0.0544,  ..., 0.1257, 0.0158, 0.0109],
        [0.1787, 0.0075, 0.0544,  ..., 0.1257, 0.0158, 0.0109],
        [0.1787, 0.0075, 0.0544,  ..., 0.1257, 0.0158, 0.0109],
        ...,
        [0.1787, 0.0075, 0.0544,  ..., 0.1257, 0.0158, 0.0109],
        [0.1787, 0.0075, 0.0544,  ..., 0.1257, 0.0158, 0.0109],
        [0.1787, 0.0075, 0.0544,  ..., 0.1257, 0.0158, 0.0109]],
       grad_fn=<ViewBackward>), tensor([[[0.0195, 0.0222, 0.0207,  ..., 0.0207, 0.0205, 0.0045],
         [0.0250, 0.0262, 0.0254,  ..., 0.0254, 0.0251, 0.0076],
         [0.0250, 0.0262, 0.0254,  ..., 0.0254, 0.0251, 0.0076],
         ...,
         [0.0348, 0.0230, 0.0231,  ..., 0.0231, 0.0243, 0.0320],
         [0.0367, 0.0240, 0.0241,  ..., 0.0241, 0.0250, 0.0330],
         [0.0355, 0.0210, 0.0211,  ..., 0.0211, 0.0215, 0.0341]],

        [[0.0195, 0.0222, 0.0207,  ..., 0.0207, 0.0205, 0.0045],
         [0.0250, 0.0262, 0.0254,  ..., 0.0254, 0.0251, 0.0076],
         [0.0250, 0.0262, 0.0254,

In [ ]:
resnet50=ResNet50(751)
resnet50.eval()
f=resnet50(imgs)

In [5]:
resnet = torchvision.models.resnet50(pretrained=True)

In [9]:
tmp=nn.Sequential(*list(resnet50.children())[:-2])

In [17]:
tmp2=list(resnet.children())[:5]

In [28]:
# 假设 tmp 是形状为 (32, 64, 16, 8) 的张量
tmp = torch.randn(32, 64, 16, 8)

# 将 tmp 进行张平操作
# 注意：在保持 batch 维度不变的情况下，对后续维度进行张平
tmp_flattened = torch.flatten(tmp, start_dim=1)

# 输出形状为 (32, 2048, 8)
print(tmp_flattened.shape)

torch.Size([32, 8192])


In [47]:
import torch

# 假设 tmp 是形状为 (32, 128, 16, 8) 的张量
tmp = torch.randn(32, 128, 16, 8)

# 合并通道维度和剩余维度，并将结果变形为 (32, 2048, 8)
tmp_merged = tmp.view(tmp.size(0), -1, tmp.size(-1))

# 添加新的维度，使其形状为 (32, 2048, 8, 1)
tmp_reshaped = tmp_merged.unsqueeze(-1)

# 输出形状为 (32, 2048, 8, 1)
print(tmp_reshaped.shape)

torch.Size([32, 2048, 8, 1])
